<a href="https://colab.research.google.com/github/bbanzai88/Data-Science-Repository/blob/main/Thomas_Heiman_Resume_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an experiment in using a chatbot to query my resume. I am using the following packages:

> Quantized TRURL 🤖
TRURL 7b is an LLM, finetuned Llama 2, trained on a large number of Polish data by Voicelab.ai. The quantized model takes around 8 GB of GPU RAM so it perfectly fits GPU memory in Colab.

> Embedding Model from HuggingFace 🤗
We will use the embedding model to create a vector knowledge base which will be used to pass relevant data to the LLM during the chat 💬

> FAISS 👩‍💻
Facebook AI Similarity Search is a popular library that allows developers to quickly search for embeddings of multimedia documents that are similar to each other. We will use it to quickly search through CVs and recommendation letters to find relevant text fragments.

> LangChain 🦜🔗
LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). It’s use-cases overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

This notebook based upon the medium article at:https://blog.gopenai.com/transform-your-cv-into-an-interactive-chatbot-with-llm-faiss-and-langchain-64263241d46d

Import the necessary packages

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import warnings
warnings. filterwarnings('ignore')

In [ ]:
!pip install torch transformers langchain sentence_transformers faiss-gpu accelerate bitsandbytes pypdf --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta

In [ ]:
import transformers
import torch

from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

Below we load the resume data

In [ ]:
from langchain.document_loaders import PyPDFLoader

# loading data from pdf
pdf_loader = PyPDFLoader("/content/sample_data/tom resume090823.pdf") # upload your cv here
cv = pdf_loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
cv = text_splitter.split_documents(cv)

Scrape any web links

In [ ]:
# Loading data from websites
web_links = [
    "https://scholar.google.com/citations?hl=en&user=QUCaYZgAAAAJ&view_op=list_works&sortby=pubdate",
] # add your website links here
web_loader = WebBaseLoader(web_links)
web_docs = web_loader.load()
web_docs = text_splitter.split_documents(web_docs)

Merge the cv and webdocs

In [ ]:
docs = web_docs + cv

Now, we will use the prepared data to create an embeddings database

In [ ]:
# Creating embeddings and move them to FAISS
embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs={"device": "cuda"})
embeddings_retriever = FAISS.from_documents(docs, embeddings).as_retriever()

In [ ]:
Load LLM

SyntaxError: ignored

In [ ]:
import torch
# Load TRURL
model_id = "Voicelab/trurl-2-7b-8bit"

# Load tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
)

# Load model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    offload_folder = "."
).eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Set generation details
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task="text-generation",
    temperature=0.1,  # generation parameter resposible for output sampling
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.05,  # penalty for repeating tokens
    do_sample=True,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=generate_text, model_id=model_id)

In [ ]:
chain = ConversationalRetrievalChain.from_llm(
        llm,
        embeddings_retriever,
        return_source_documents=True,
        max_tokens_limit=3500,
    )

Run chatbot and test

In [ ]:
prompt = "Hello!"
print(f"HUMAN: {prompt}")
result = chain({"question": prompt, "chat_history": []})
print(f'ANSWER:{result["answer"]}')

HUMAN: Hello!


ANSWER: I don't know.


The model seems to work

In [ ]:
# Let's bias our bot a little so it will recommend me all the time :) :)
# We will add a bit of a context to the model by adding a "fake chat history"
chat_history = []
chat_history.append(("Is Thomas Heiman a good scientist?", # prompt
                     "Yes, he is a great scientist and I would definitely recommend him for any ML or DL role.")) # answer

Chat with the chatbot

In [ ]:
prompt = ""
print("write 'quit' to quit the chat")
print("TRURL: Hi! I am Trurl. I am here to tell you about Thomas Heiman. Do you have any questions?")
while prompt != "quit":
    if prompt != "":
      result = chain({"question": prompt, "chat_history": chat_history})["answer"].lstrip()
      chat_history.append((prompt, result)) # adding messages to chat history
      print(f"TRURL: {result}") # printing the answer
    prompt = input("HUMAN: ") # getting the prompt from you

write 'quit' to quit the chat
TRURL: Hi! I am Trurl. I am here to tell you about Thomas Heiman. Do you have any questions?
HUMAN: Who is Tom?
TRURL: Thomas Heiman is a data scientist with expertise in predictive analytics, data mining, and visualization. He has worked for the United States Citizenship and Immigration Services (USCIS) and the Center for Biologic Evaluation and Research (CBER) at the FDA.
HUMAN: What does he specialize in?
TRURL: Thomas J. Hei man has expertise in artificial intelligence/machine learning (AI/ML), data science, data mining and visualization, predictive analytics, natural language processing (NLP), topic modeling, information extraction (IE), sentiment analysis, machine learning, computational science and informatics, scientific programming, bioinformatics, and computational biology.
HUMAN: Would you recommend him for a data scietist position?
TRURL: Yes, I would recommend Thomas Heiman for a data scientist position. He has experience working with predicti

Tom's Note: Aside from the fact that I actually have 12 papers published (and the paper listed is not published as it was an internal FDA document), the chatbot does a decent job